import Pkg; Pkg.add("FFTW")
import Pkg; Pkg.add("PaddedViews")
import Pkg; Pkg.add("BenchmarkTools")

In [1]:
using FFTW
using Base.Threads
using PaddedViews
using BenchmarkTools

#try out different fft planners https://docs.juliahub.com/FourierAnalysis/1aWDG/1.0.1/fftw/
#also switch to in-place operations at some point
function create_2Dffp_plans(n::Int64,nthr::Int64)
    FFTW.set_num_threads(nthr)
    plan_fwd = plan_fft(rand(ComplexF64, n,n),flags=FFTW.MEASURE)
    plan_bwd = plan_ifft(rand(ComplexF64,n,n),flags=FFTW.MEASURE)
    (plan_fwd,plan_bwd)
end

function fftconv2d(arr::Array{ComplexF64,2},fftker::Array{ComplexF64,2},plan_fwd,plan_bwd,fwd::Bool=true)::Array{ComplexF64,2}
    narr = size(arr)[1]
    nker = size(fftker)[1]
    nout = nker - narr
    if fwd==true
        i1,i2 = 1+narr÷2, narr÷2+nout
    else
        i1,i2 =   narr÷2, narr÷2+nout-1
    end
    ifftshift(plan_bwd * (fftker .* (plan_fwd * sym_paddedviews(0.0+im*0.0,arr,fftker)[1])))[i1:i2,i1:i2]
end
    

fftconv2d (generic function with 2 methods)

In [2]:
arr = rand(ComplexF64,5000,5000)
fftker = rand(ComplexF64,8000,8000)

(p1,p2) = create_2Dffp_plans(8000,2); @btime outarr = fftconv2d(arr,fftker,p1,p2,true);
(p1,p2) = create_2Dffp_plans(8000,4); @btime outarr = fftconv2d(arr,fftker,p1,p2,true);

  2.788 s (139 allocations: 4.90 GiB)
  2.426 s (194 allocations: 4.90 GiB)
